# SEGMENTING AND CLUSTERING NEIGHBORHOODS OF TORONTO

### 1 Importing Libraries

In [1]:
import numpy as np            # for efficient numerical operations
import pandas as pd           # for data frame analytics
import requests               # for pulling html pages and creating html documents
from bs4 import BeautifulSoup # for scraping html documents

### 2 Parsing the Wikipedia page on Toronto postal codes

By means of the <i>requests</i> library which has been loaded above, the link provided in the description of the assignment can be employed to retrieve the html page pointed to be that link. This is acchieved via the <i>requests.get()</i> function. Once the html page is retrieved the underlying html document stored in the <i>text</i> attribute of the page object. Lastly, this hmtl document is parsed by means of the <i>BeautifulSoup</i> library.

In [2]:
html_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_page = requests.get(html_link)
html_doc = html_page.text
soup = BeautifulSoup(html_doc,'html.parser')

### 3 Scraping the html document

#### 3.1 Searching the postal code table in the html document and creating a dataframe from it
Firstly, the soup object is searched for the table by means of the <i>find()</i> function and specifying the html tag <i>table</i>. With the help of that same function the rows (html tagged <i>tr</i>) are then extracted from the table, and headers (html tagged <i>th</i>) are extracted from the first row. These headers, together with the number of rows are employed to create a raw pandas dataframe <i>table_df</i> which contains all rows of the table found on the wikipedia page.

In [3]:
table = soup.find('table')                                                   # searching for table in html document
rows = table.findAll('tr')                                                   # gathering all rows in that table
header = rows[0].findAll('th')                                               # gathering all column headers in first row of the table
col_num = len(header)                                                        # number of columns equals number of headers found and hence the length of the header object
row_num = len(rows)-1                                                        # number of rows equals number of table rows minus one since first row did contain column headers

columns = list()                                                             # writing column headers into a list
for c in range(col_num):
    if c != col_num-1:
        columns.append(header[c].string.lower())
    else:
        columns.append(header[c].string[:-1].lower())

table_df = pd.DataFrame(columns=columns,index=range(row_num))                # creating a pandas data frame with column names defined by the list of column headers and index defined by number of rows

for i in range(row_num):                                                     # filling the table row by row
    row = rows[i+1].findAll('td')
    for c in range(col_num):
        table_df.iloc[i,c] = row[c].string
        if table_df.iloc[i,c] == None:                                       # the entry of the wikipedia table may have been a string or a hyperlink. If the latter the td object does not provide the string
            table_df.iloc[i,c] = row[c].a.string                             # but its child 'a' does

for i in table_df.index:                                                     # removing line breaks
    if table_df.loc[i,'neighbourhood'][-1:] == '\n':
        table_df.loc[i,'neighbourhood']=table_df.loc[i,'neighbourhood'][:-1]
        
table_df.head()

,postcode,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 3.2 Manipulating the raw dataframe
Next, those rows of the raw dataframe <i>table_df</i> data frame who have no borough assigned, are dropped. Next, if among the remaining entries the neighborhood is not assigned, the neighborhood name will be the same as the borough name.

In [4]:
for i in table_df.index:
    if table_df.loc[i,'borough'] == 'Not assigned':
        table_df = table_df.drop(i,axis=0)
    elif table_df.loc[i,'neighbourhood'] == 'Not assigned':
        table_df.loc[i,'neighbourhood'] = table_df.loc[i,'borough']
        
table_df.head()

,postcode,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### 3.3  Creating a clean postal code dataframe from the raw data frame
Lastly, from the raw dataframe <i>table_df</i> the unique postal codes are collected and employed to create a clean dataframe <i>code_df</i> which lists codes, borough names and all associated neighborhoods.

In [5]:
code_list = list(table_df['postcode'].unique())
code_df = pd.DataFrame(columns=columns,index=range(len(code_list)))

for i in code_df.index:
    code = code_list[i]
    sub_df = table_df[table_df['postcode']==code]
    hood_string = ''
    for j in range(sub_df.shape[0]):
        hood_string = hood_string + sub_df.loc[sub_df.index[j],'neighbourhood'] + ', '
    hood_string = hood_string[:-2]
    code_df.loc[i,'postcode'] = code
    code_df.loc[i,'borough'] = sub_df.loc[sub_df.index[0],'borough']
    code_df.loc[i,'neighbourhood'] = hood_string
    
code_df.sort_values('postcode',inplace=True)
code_df.reset_index(inplace=True)
code_df.drop('index',axis=1,inplace=True)
    
code_df.head()

,postcode,borough,neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally, the number of rows of the <i>code_df</i> dataframe can be infered from its <i>shape[0]</i> object.

In [6]:
print('Number of postcodes: ',code_df.shape[0])

Number of postcodes:  103


### 4 Assigning latitude and longitude

Since the geocoder strategy did not work, the csv file containing the georeferences for the Toronto postal codes is loaded into a dataframe which is then employed in order to look up the postal codes needed.

In [7]:
geodata_df = pd.read_csv('https://cocl.us/Geospatial_data') # loading geodata in a pandas dataframe
geodata_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
code_df['latitude'] = ''                                                               # adding latitude column to code_df data frame
code_df['longitude'] = ''                                                              # adding longitude column to code_df data frame

for i in code_df.index:
    code = code_df.loc[i,'postcode']
    code_df.loc[i,'latitude'] = geodata_df[geodata_df['Postal Code']==code].iloc[0,1]  # assigning latidtude by reduceing geodata_df to the entry matching the postal code and then calling its latitude column 1
    code_df.loc[i,'longitude'] = geodata_df[geodata_df['Postal Code']==code].iloc[0,2] # assigning longidtude by reduceing geodata_df to the entry matching the postal code and then calling its longitude column 2
    
code_df.head()

,postcode,borough,neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
